In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import keras
from keras.layers import Embedding, Masking, Concatenate, GRU, Dense, Reshape
from model import CNN_with_mask
from other_model import prosenet_model, bilstm, onlstm

In [4]:
def get_padding_data(df):
    result_col = ['Errors', 'Score', 'Nothing']
    col = [c for c in df.columns if c not in result_col and c != 'Game' and c != 'Rally']

    team_col = [c for c in df.columns if 'Team' in c]

    rally_set = []
    rally_result_set = []
    for _, df_rally in df.groupby(['Game', 'Rally']):   # each rally in one game
        # print(df_rally)
        curr_team = df.iloc[df_rally.index[0]][team_col].tolist()
        shot_set = []
        shot_result_set = []
        
        atk_sequence = []
        atk_result = []
        
        for _, shot in df_rally.iterrows():
            if(shot[team_col].tolist() != curr_team):
                shot_set.append(atk_sequence)
                shot_result_set.append(atk_result)   # 最後一動的結果 -> predict object
                
                curr_team = shot[team_col].tolist()

                atk_sequence = []

            atk_sequence.append(shot[col])
            atk_result = shot[result_col]
        
        # the last shot
        shot_set.append(atk_sequence)
        shot_result_set.append(atk_result)

        # one rally has been finished
        shot_set = pad_sequences(shot_set, maxlen=3, padding='post')

        # one rally has been finished
        rally_set.append(shot_set)
        rally_result_set.append(shot_result_set)

    padded_rally_set = pad_sequences(rally_set, dtype=float, padding='post')
    padded_rally_result_set = pad_sequences(rally_result_set, dtype=float, padding='post')
    
    return padded_rally_set, padded_rally_result_set


In [5]:
df = pd.read_csv('test.csv')
df = pd.get_dummies(df)

In [6]:
rally_set, rally_result_set = get_padding_data(df)

In [7]:
# rally數, 最大回合數in one rally, 3, feature數
print(rally_set.shape)
print(rally_result_set.shape)

(1292, 14, 3, 43)
(1292, 14, 3)


In [17]:
# rally_result_set = rally_result_set.reshape(1292, 2)

ValueError: cannot reshape array of size 54264 into shape (1292,2)

In [8]:
rally_set_tensor = tf.convert_to_tensor(rally_set)
rally_result_set_tensor = tf.convert_to_tensor(rally_result_set)

rally_result_set_tensor = tf.where(tf.math.is_nan(rally_result_set_tensor), 0.0, rally_result_set_tensor)

In [9]:
def split_data(others_tensor, label_tensor):
    l = label_tensor.shape[0]
    split_persentage = int(l*0.7)

    train_x = others_tensor[:split_persentage]
    train_y = label_tensor[:split_persentage]

    test_x = others_tensor[split_persentage:]
    test_y = label_tensor[split_persentage:]

    return train_x, train_y, test_x, test_y

In [10]:
train_x, train_y, test_x, test_y = split_data(rally_set_tensor, rally_result_set_tensor)

In [22]:
model = prosenet_model(shot_sequence_shape=rally_set.shape[1:], nclasses=3)
model.summary()

Model: "Classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Shots_input (InputLayer)     [(None, 14, 3, 43)]       0         
_________________________________________________________________
Sequence_masking (Masking)   (None, 14, 3, 43)         0         
_________________________________________________________________
cnn_with_mask_1 (CNN_with_ma (None, 14, 1, 43)         5590      
_________________________________________________________________
tf.compat.v1.squeeze_1 (TFOp (None, 14, 43)            0         
_________________________________________________________________
classifier (Dense)           (None, 14, 3)             129       
Total params: 5,719
Trainable params: 5,719
Non-trainable params: 0
_________________________________________________________________


In [23]:
# regularizer = tf.keras.regularizers.l2(0.01)
optimizer = 'adam'
loss = keras.losses.CategoricalCrossentropy()
metrics = ['accuracy']
epochs = 30
# callbacks = tf.keras.callbacks.EarlyStopping(min_delta=0.002, patience=15, restore_best_weights=True)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [24]:
model.fit(train_x, train_y, epochs=30)

Epoch 1/30
29/29 [==============================] - 0s 890us/step - loss: 0.2166 - accuracy: 0.1427
Epoch 2/30
29/29 [==============================] - 0s 926us/step - loss: 0.2032 - accuracy: 0.1329
Epoch 3/30
29/29 [==============================] - 0s 962us/step - loss: 0.1945 - accuracy: 0.1399
Epoch 4/30
29/29 [==============================] - 0s 855us/step - loss: 0.1884 - accuracy: 0.1428
Epoch 5/30
29/29 [==============================] - 0s 926us/step - loss: 0.1841 - accuracy: 0.1425
Epoch 6/30
29/29 [==============================] - 0s 855us/step - loss: 0.1808 - accuracy: 0.1448
Epoch 7/30
29/29 [==============================] - 0s 873us/step - loss: 0.1792 - accuracy: 0.1451
Epoch 8/30
29/29 [==============================] - 0s 962us/step - loss: 0.1774 - accuracy: 0.1455
Epoch 9/30
29/29 [==============================] - 0s 855us/step - loss: 0.1751 - accuracy: 0.1455
Epoch 10/30
29/29 [==============================] - 0s 1ms/step - loss: 0.1736 - accuracy: 0.1454
E

In [17]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

In [27]:
result = model.evaluate(test_x, test_y)

13/13 [==============================] - 0s 748us/step - loss: 0.1728 - accuracy: 0.1443


In [29]:
y_pred = model.predict(test_x)

test_y_reshape = test_y.reshape(test_y.shape[0]*test_y.shape[1], 3)
y_pred_reshape = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], 3)

idx = [i for i in range(len(test_y_reshape)) if(any(test_y_reshape[i] == np.array([1, 1, 1])))]
print(len(idx))

argmax_y_pred_reshape = np.argmax(y_pred_reshape[idx], axis=1)
argmax_test_y_reshape = np.argmax(test_y_reshape[idx], axis=1)

1172


In [30]:
from sklearn.metrics import accuracy_score, roc_auc_score
from evaluate import calculate_BS, f1, show_eval_result

acc_score = accuracy_score(argmax_test_y_reshape, argmax_y_pred_reshape)
f1_score = f1(argmax_test_y_reshape, argmax_y_pred_reshape)
auc_score = roc_auc_score(test_y_reshape[idx, :], y_pred_reshape[idx, :], multi_class='ovr')
BS = calculate_BS(test_y_reshape[idx, :], y_pred_reshape[idx, :], 3)

show_eval_result(acc_score, f1_score, auc_score, BS)

accuracy:    0.67
f1:          0.85
auc:         0.81
BS:          0.16, 0.12, 0.19
mean of BS:  0.15


---

In [14]:
model = bilstm(shot_sequence_shape=rally_set.shape[1:])
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Shots_input (InputLayer)     [(None, 14, 3, 43)]       0         
_________________________________________________________________
Sequence_masking (Masking)   (None, 14, 3, 43)         0         
_________________________________________________________________
cnn_with_mask_2 (CNN_with_ma (None, 14, 1, 43)         5590      
_________________________________________________________________
tf.compat.v1.squeeze_2 (TFOp (None, 14, 43)            0         
_________________________________________________________________
Bidirectional_recurrent_laye (None, 14, 64)            19456     
_________________________________________________________________
dense_2 (Dense)              (None, 14, 3)             195       
Total params: 25,241
Trainable params: 25,241
Non-trainable params: 0
_______________________________________________________

In [15]:
# regularizer = tf.keras.regularizers.l2(0.01)
optimizer = 'adam'
loss = keras.losses.CategoricalCrossentropy()
metrics = ['accuracy']
epochs = 30
# callbacks = tf.keras.callbacks.EarlyStopping(min_delta=0.002, patience=15, restore_best_weights=True)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [16]:
model.fit(train_x, train_y, epochs=epochs)

Epoch 1/30
29/29 [==============================] - 2s 4ms/step - loss: 0.1950 - accuracy: 0.1830
Epoch 2/30
29/29 [==============================] - 0s 4ms/step - loss: 0.1523 - accuracy: 0.7138
Epoch 3/30
29/29 [==============================] - 0s 4ms/step - loss: 0.1491 - accuracy: 0.9490
Epoch 4/30
29/29 [==============================] - 0s 4ms/step - loss: 0.1414 - accuracy: 0.8998
Epoch 5/30
29/29 [==============================] - 0s 4ms/step - loss: 0.1348 - accuracy: 0.8918
Epoch 6/30
29/29 [==============================] - 0s 4ms/step - loss: 0.1301 - accuracy: 0.9501
Epoch 7/30
29/29 [==============================] - 0s 4ms/step - loss: 0.1267 - accuracy: 0.9696
Epoch 8/30
29/29 [==============================] - 0s 4ms/step - loss: 0.1230 - accuracy: 0.9712
Epoch 9/30
29/29 [==============================] - 0s 4ms/step - loss: 0.1197 - accuracy: 0.9723
Epoch 10/30
29/29 [==============================] - 0s 4ms/step - loss: 0.1166 - accuracy: 0.9727
Epoch 11/30
29/29 [

In [17]:
result = model.evaluate(test_x, test_y)

13/13 [==============================] - 0s 1ms/step - loss: 0.0918 - accuracy: 0.9720


In [20]:
y_pred = model.predict(test_x)

test_y_reshape = test_y.reshape(test_y.shape[0]*test_y.shape[1], 3)
y_pred_reshape = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], 3)

idx = [i for i in range(len(test_y_reshape)) if(any(test_y_reshape[i] == np.array([1, 1, 1])))]
print(len(idx))

argmax_y_pred_reshape = np.argmax(y_pred_reshape[idx], axis=1)
argmax_test_y_reshape = np.argmax(test_y_reshape[idx], axis=1)

1172


In [21]:
from sklearn.metrics import accuracy_score, roc_auc_score
from evaluate import calculate_BS, f1, show_eval_result

acc_score = accuracy_score(argmax_test_y_reshape, argmax_y_pred_reshape)
f1_score = f1(argmax_test_y_reshape, argmax_y_pred_reshape)
auc_score = roc_auc_score(test_y_reshape[idx, :], y_pred_reshape[idx, :], multi_class='ovr')
BS = calculate_BS(test_y_reshape[idx, :], y_pred_reshape[idx, :], 3)

show_eval_result(acc_score, f1_score, auc_score, BS)

accuracy:    0.87
f1:          0.92
auc:         0.50
BS:          0.33, 0.21, 0.46
mean of BS:  0.33


---

In [22]:
!pip install keras_ordered_neurons

  Created wheel for keras-ordered-neurons: filename=keras_ordered_neurons-0.10.0-py3-none-any.whl size=10963 sha256=b22155be9b78f1f5f6308d57e85840ce8885f5c3fba3fee3bc0f2025c28ab2dd
  Stored in directory: c:\users\yichen\appdata\local\pip\cache\wheels\04\6e\db\c16295f3a0bd5a22579aca2e00e1d51ee37e259248d2eb4a46
Successfully built keras-ordered-neurons


In [12]:
model = onlstm(shot_sequence_shape=rally_set.shape[1:])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Shots_input (InputLayer)     [(None, 14, 3, 43)]       0         
_________________________________________________________________
Sequence_masking (Masking)   (None, 14, 3, 43)         0         
_________________________________________________________________
cnn_with_mask_1 (CNN_with_ma (None, 14, 1, 43)         5590      
_________________________________________________________________
tf.compat.v1.squeeze_1 (TFOp (None, 14, 43)            0         
_________________________________________________________________
ONLSTM (ONLSTM)              (None, 14, 32)            10944     
_________________________________________________________________
dense_1 (Dense)              (None, 14, 3)             99        
Total params: 16,633
Trainable params: 16,633
Non-trainable params: 0
_______________________________________________________

In [24]:
from typing import Tuple, Dict, Any

In [35]:
def eval_model_result(train_x, train_y, test_x, test_y, 
                        model_kwargs: Dict[str, Any] = {'model': None, 'optimizer': None, 'loss': None, 'metrics': None, 'epochs': None, 'shapes': []}):
    if(model_kwargs['model'] == 'prosenet_model'):
        pass
    elif(model_kwargs['model'] == 'deepmoji'):
        pass
    elif(model_kwargs['model'] == 'bilstm'):
        pass
    elif(model_kwargs['model'] == 'onlstm'):
        pass
    elif(model_kwargs['model'] == 'transformer'):
        pass

In [37]:
eval_model_result(train_x, train_y, test_x, test_y, 
                  model_kwargs={'model': 'onlstm', 
                                'optimizer': 'adam', 
                                'loss': keras.losses.CategoricalCrossentropy(), 
                                'metrics': ['accuracy'], 
                                'epochs': 30, 
                                'shapes': [10, 3, 24]})

onlstm
adam
['accuracy']


In [13]:
# regularizer = tf.keras.regularizers.l2(0.01)
optimizer = 'adam'
loss = keras.losses.CategoricalCrossentropy()
metrics = ['accuracy']
epochs = 30
# callbacks = tf.keras.callbacks.EarlyStopping(min_delta=0.002, patience=15, restore_best_weights=True)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [14]:
model.fit(train_x, train_y, epochs=30)

Epoch 1/30
29/29 [==============================] - 2s 8ms/step - loss: 0.2304 - accuracy: 0.2447
Epoch 2/30
29/29 [==============================] - 0s 7ms/step - loss: 0.2215 - accuracy: 0.1800
Epoch 3/30
29/29 [==============================] - 0s 7ms/step - loss: 0.2151 - accuracy: 0.1323
Epoch 4/30
29/29 [==============================] - 0s 7ms/step - loss: 0.2116 - accuracy: 0.1114
Epoch 5/30
29/29 [==============================] - 0s 7ms/step - loss: 0.2105 - accuracy: 0.1056
Epoch 6/30
29/29 [==============================] - 0s 7ms/step - loss: 0.2105 - accuracy: 0.1059
Epoch 7/30
29/29 [==============================] - 0s 7ms/step - loss: 0.2111 - accuracy: 0.1056
Epoch 8/30
29/29 [==============================] - 0s 7ms/step - loss: 0.2110 - accuracy: 0.1060
Epoch 9/30
29/29 [==============================] - 0s 7ms/step - loss: 0.2106 - accuracy: 0.1067
Epoch 10/30
29/29 [==============================] - 0s 7ms/step - loss: 0.2109 - accuracy: 0.1078
Epoch 11/30
29/29 [

In [15]:
result = model.evaluate(test_x, test_y)

13/13 [==============================] - 0s 2ms/step - loss: 0.2132 - accuracy: 0.1079


In [18]:
y_pred = model.predict(test_x)

test_y_reshape = test_y.reshape(test_y.shape[0]*test_y.shape[1], 3)
y_pred_reshape = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], 3)

idx = [i for i in range(len(test_y_reshape)) if(any(test_y_reshape[i] == np.array([1, 1, 1])))]
print(len(idx))

argmax_y_pred_reshape = np.argmax(y_pred_reshape[idx], axis=1)
argmax_test_y_reshape = np.argmax(test_y_reshape[idx], axis=1)

1172


In [19]:
from sklearn.metrics import accuracy_score, roc_auc_score
from evaluate import calculate_BS, f1, show_eval_result

acc_score = accuracy_score(argmax_test_y_reshape, argmax_y_pred_reshape)
f1_score = f1(argmax_test_y_reshape, argmax_y_pred_reshape)
auc_score = roc_auc_score(test_y_reshape[idx, :], y_pred_reshape[idx, :], multi_class='ovr')
BS = calculate_BS(test_y_reshape[idx, :], y_pred_reshape[idx, :], 3)

show_eval_result(acc_score, f1_score, auc_score, BS)

accuracy:    0.50
f1:          0.81
auc:         0.49
BS:          0.33, 0.21, 0.46
mean of BS:  0.33


---